The aim of this notebook is to obtain the carbon emission of each region, and merge this data to the already preprocessed one called ``df_process.csv``. We only have the emmissions for the whole country from 2019 to 2023, and the emmissions per region for 2019 and 2020, so we will calculate weigths for each region between 2019 and 2020 and apply them to the 2019-2023 data. The emission time step is on a daily basis, so we will synchronize the already processed data with the emmission data. Finally, we also add to this dataset the regular and thermosensitive part of the energy curve, for both electricity and gaz. We save the obtained dataset in ``carbon_data.csv``.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
pd.options.plotting.backend = "plotly"

from energydisaggregation.models.stat_model import (
    Stats,
    preprocess,
    split_year,
)

## Data processsing

### Collect source variables (Date, Région, Temperature)

In [30]:
# Import data
df_src = pd.read_csv(
    "EnergyDisaggregation/energydisaggregation/data_storage/df_process.csv",
    sep=",",
)
# df_src.info()
df_src.columns

Index(['Date - Heure', 'Région', 'consommation brute électricité (mw) - rte',
       'température (°c)', 'nebulosité totale', 'vitesse du vent moyen 10 mn',
       'humidité', 'vitesse du vent en km/h', 'température ressentie',
       'saison', 'week_day', 'month', 'hour', 'is_holiday', 'is_bank_holiday',
       'day_of_year', 'years', 'regions', 'temperature_seuil', 'saturation',
       'diff_seuil', 'température ressentie.1', 'température (°c)_mean_48',
       'température (°c)_std_48', 'température (°c)_max_48',
       'température (°c)_min_48', 'nebulosité totale_mean_3',
       'nebulosité totale_std_3', 'month_sin', 'month_cos', 'week_day_sin',
       'week_day_cos', 'hour_sin', 'hour_cos',
       'consommation brute électricité (mw) - rte_lag_1',
       'consommation brute électricité (mw) - rte_lag_2',
       'consommation brute électricité (mw) - rte_lag_3',
       'consommation brute électricité (mw) - rte_lag_4',
       'consommation brute électricité (mw) - rte_lag_5',
    

In [31]:
vars_selected = [
    "Date - Heure",
    "Région",
    "saison",
    "week_day",
    "is_holiday",
    "is_bank_holiday",
    "consommation brute électricité (mw) - rte",
    "consommation brute électricité (mw) - rte_lag_1",
    "consommation brute électricité (mw) - rte_lag_2",
    "consommation brute électricité (mw) - rte_lag_3",
    "consommation brute électricité (mw) - rte_lag_4",
    "consommation brute électricité (mw) - rte_lag_5",
    "consommation brute électricité (mw) - rte_lag_6",
    "consommation brute électricité (mw) - rte_lag_7",
    "température (°c)",
    "température (°c)_lag_1",
    "température (°c)_lag_2",
    "température (°c)_lag_3",
    "température (°c)_lag_4",
    "température (°c)_lag_5",
    "température (°c)_lag_6",
    "température (°c)_lag_7",
]

key_vars = ["Date", "Region"]

values_var = [
    "saison",
    "week_day",
    "is_holiday",
    "is_bank_holiday",
    "ce",
    "ce1",
    "ce2",
    "ce3",
    "ce4",
    "ce5",
    "ce6",
    "ce7",
    "Temp",
    "Temp1",
    "Temp2",
    "Temp3",
    "Temp4",
    "Temp5",
    "Temp6",
    "Temp7",
]

tot_vars = key_vars + values_var

renamed_vars = dict(zip(vars_selected, tot_vars))
renamed_vars

{'Date - Heure': 'Date',
 'Région': 'Region',
 'saison': 'saison',
 'week_day': 'week_day',
 'is_holiday': 'is_holiday',
 'is_bank_holiday': 'is_bank_holiday',
 'consommation brute électricité (mw) - rte': 'ce',
 'consommation brute électricité (mw) - rte_lag_1': 'ce1',
 'consommation brute électricité (mw) - rte_lag_2': 'ce2',
 'consommation brute électricité (mw) - rte_lag_3': 'ce3',
 'consommation brute électricité (mw) - rte_lag_4': 'ce4',
 'consommation brute électricité (mw) - rte_lag_5': 'ce5',
 'consommation brute électricité (mw) - rte_lag_6': 'ce6',
 'consommation brute électricité (mw) - rte_lag_7': 'ce7',
 'température (°c)': 'Temp',
 'température (°c)_lag_1': 'Temp1',
 'température (°c)_lag_2': 'Temp2',
 'température (°c)_lag_3': 'Temp3',
 'température (°c)_lag_4': 'Temp4',
 'température (°c)_lag_5': 'Temp5',
 'température (°c)_lag_6': 'Temp6',
 'température (°c)_lag_7': 'Temp7'}

In [32]:
df_X = df_src.copy()

df_X = df_X[vars_selected]

# We truncate "Date - Heure" so that we only keep the Date
def truncate(s, start=0, end=10):
    return s[start:end]

var = "Date - Heure"
res_map = map(truncate, df_src[var])
date_truncated = pd.Series(np.array(list(res_map)))
date_truncated = pd.to_datetime(date_truncated, format="%Y-%m-%d")
df_X[var] = date_truncated

In [33]:
# On renomme les variables
df_X = df_X.rename(columns=renamed_vars)
df_X

,Date,Region,saison,week_day,is_holiday,is_bank_holiday,ce,ce1,ce2,ce3,...,ce6,ce7,Temp,Temp1,Temp2,Temp3,Temp4,Temp5,Temp6,Temp7
0,2013-01-01,Auvergne-Rhône-Alpes,3,1,True,True,8173.0,NaN,NaN,NaN,...,NaN,NaN,9.375000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2013-01-01,Bourgogne-Franche-Comté,3,1,True,True,2357.0,NaN,NaN,NaN,...,NaN,NaN,2.675000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013-01-01,Bretagne,3,1,True,True,3050.0,NaN,NaN,NaN,...,NaN,NaN,9.475000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2013-01-01,Centre-Val de Loire,3,1,True,True,2476.0,NaN,NaN,NaN,...,NaN,NaN,8.200000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2013-01-01,Grand Est,3,1,True,True,4943.0,NaN,NaN,NaN,...,NaN,NaN,5.075000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
736123,2019-12-31,Nouvelle-Aquitaine,3,1,True,False,12110.0,555.0,339.0,-248.0,...,-362.0,-456.0,8.133333,-0.133333,-0.216667,-0.283333,-0.350000,-0.666667,-1.066667,-1.466667
736124,2019-12-31,Occitanie,3,1,True,False,10984.0,786.0,441.0,-112.0,...,734.0,1017.0,5.000000,-0.800000,-1.650000,-2.516667,-3.383333,-4.375000,-5.408333,-6.441667
736125,2019-12-31,Pays de la Loire,3,1,True,False,8133.0,977.0,920.0,517.0,...,25.0,39.0,7.525000,0.100000,0.125000,0.125000,0.125000,-0.175000,-0.575000,-0.975000
736126,2019-12-31,Provence-Alpes-Côte d'Azur,3,1,True,False,12127.0,971.0,788.0,226.0,...,802.0,1214.0,4.450000,2.200000,1.225000,-0.808333,-2.841667,-4.425000,-5.858333,-7.291667


In [34]:
# On sépare les variables de consommation des autres variables
conso_vars = np.array(tot_vars)[(np.array(tot_vars) >= "ce") * (np.array(tot_vars) <= "ce9")]

# On génère les datasets consommation VS autres variables
df_conso = df_X[key_vars].join(df_X[conso_vars])
df_not_conso = df_X.drop(columns=conso_vars)

# Il faut sommer la consommation au lieu de la moyenner
# Tandis qu'on moyenne des autres variables telles que la température
df_conso = df_conso.groupby(key_vars).sum()
df_not_conso = df_not_conso.groupby(key_vars).mean()

# On joint les deux datasets
df_X = df_not_conso.join(df_conso, on=key_vars)
df_X

saison  week_day  is_holiday  \
Date       Region                                                     
2013-01-01 Auvergne-Rhône-Alpes           3.0       1.0         1.0   
           Bourgogne-Franche-Comté        3.0       1.0         1.0   
           Bretagne                       3.0       1.0         1.0   
           Centre-Val de Loire            3.0       1.0         1.0   
           Grand Est                      3.0       1.0         1.0   
...                                       ...       ...         ...   
2019-12-31 Nouvelle-Aquitaine             3.0       1.0         1.0   
           Occitanie                      3.0       1.0         1.0   
           Pays de la Loire               3.0       1.0         1.0   
           Provence-Alpes-Côte d'Azur     3.0       1.0         1.0   
           Île-de-France                  3.0       1.0         1.0   

                                       is_bank_holiday      Temp     Temp1  \
Date       Region                                                            
2013-01-01 Auvergne-Rhône-Alpes                    1.0  6.137500 -0.307609   
           Bourgogne-Franche-Comté                 1.0  5.512500  0.015217   
           Bretagne                                1.0  8.370833 -0.092391   
           Centre-Val de Loire                     1.0  7.590625 -0.211594   
           Grand Est                               1.0  6.761458  0.010870   
...                                                ...       ...       ...   
2019-12-31 Nouvelle-Aquitaine                      0.0  5.902083  0.355556   
           Occitanie                               0.0  6.320833 -0.044792   
           Pays de la Loire                        0.0  5.019792  0.272569   
           Provence-Alpes-Côte d'Azur              0.0  7.189583  0.093403   
           Île-de-France                           0.0  0.692708 -0.071528   

                                          Temp2     Temp3     Temp4     Temp5  \
Date       Region                                                               
2013-01-01 Auvergne-Rhône-Alpes       -0.613258 -0.846429 -0.992917 -1.068421   
           Bourgogne-Franche-Comté     0.034848  0.047619  0.058333  0.074123   
           Bretagne                   -0.107197 -0.144048 -0.237917 -0.390789   
           Centre-Val de Loire        -0.428030 -0.636905 -0.828333 -0.997807   
           Grand Est                   0.029167  0.036905  0.032083  0.016667   
...                                         ...       ...       ...       ...   
2019-12-31 Nouvelle-Aquitaine          0.700000  0.989583  1.209722  1.360417   
           Occitanie                  -0.127083 -0.142708 -0.056944  0.130208   
           Pays de la Loire            0.543750  0.787500  0.995139  1.166667   
           Provence-Alpes-Côte d'Azur  0.152083  0.189583  0.210417  0.214583   
           Île-de-France              -0.162500 -0.243750 -0.305556 -0.347917   

                                          Temp6     Temp7        ce     ce1  \
Date       Region                                                             
2013-01-01 Auvergne-Rhône-Alpes       -1.169444 -1.305882  355359.0  8080.0   
           Bourgogne-Franche-Comté     0.113889  0.179902  110467.0  2561.0   
           Bretagne                   -0.505556 -0.545588  121979.0  2957.0   
           Centre-Val de Loire        -1.133333 -1.227941  103218.0  2252.0   
           Grand Est                  -0.001389 -0.023529  219318.0  4994.0   
...                                         ...       ...       ...     ...   
2019-12-31 Nouvelle-Aquitaine          1.493750  1.627083  306332.0  -922.0   
           Occitanie                   0.289583  0.378125  255352.0  -342.0   
           Pays de la Loire            1.273958  1.307639  197521.0  -644.0   
           Provence-Alpes-Côte d'Azur  0.251042  0.336111  273760.0    54.0   
           Île-de-France              -0.415625 -0.523611  504624.0  -113.0   

                  

In [35]:
df_X = df_X.reset_index()
df_X

,Date,Region,saison,week_day,is_holiday,is_bank_holiday,Temp,Temp1,Temp2,Temp3,...,Temp6,Temp7,ce,ce1,ce2,ce3,ce4,ce5,ce6,ce7
0,2013-01-01,Auvergne-Rhône-Alpes,3.0,1.0,1.0,1.0,6.137500,-0.307609,-0.613258,-0.846429,...,-1.169444,-1.305882,355359.0,8080.0,7785.0,7945.0,8964.0,10785.0,12438.0,13480.0
1,2013-01-01,Bourgogne-Franche-Comté,3.0,1.0,1.0,1.0,5.512500,0.015217,0.034848,0.047619,...,0.113889,0.179902,110467.0,2561.0,3124.0,3617.0,4259.0,5229.0,6097.0,6609.0
2,2013-01-01,Bretagne,3.0,1.0,1.0,1.0,8.370833,-0.092391,-0.107197,-0.144048,...,-0.505556,-0.545588,121979.0,2957.0,2892.0,2781.0,3495.0,4717.0,5800.0,6148.0
3,2013-01-01,Centre-Val de Loire,3.0,1.0,1.0,1.0,7.590625,-0.211594,-0.428030,-0.636905,...,-1.133333,-1.227941,103218.0,2252.0,2334.0,2556.0,3052.0,3875.0,4626.0,4787.0
4,2013-01-01,Grand Est,3.0,1.0,1.0,1.0,6.761458,0.010870,0.029167,0.036905,...,-0.001389,-0.023529,219318.0,4994.0,5317.0,5601.0,6656.0,8336.0,9974.0,10975.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30667,2019-12-31,Nouvelle-Aquitaine,3.0,1.0,1.0,0.0,5.902083,0.355556,0.700000,0.989583,...,1.493750,1.627083,306332.0,-922.0,-1842.0,-2740.0,-3589.0,-4140.0,-3853.0,-2931.0
30668,2019-12-31,Occitanie,3.0,1.0,1.0,0.0,6.320833,-0.044792,-0.127083,-0.142708,...,0.289583,0.378125,255352.0,-342.0,-778.0,-1235.0,-1786.0,-2176.0,-2255.0,-2175.0
30669,2019-12-31,Pays de la Loire,3.0,1.0,1.0,0.0,5.019792,0.272569,0.543750,0.787500,...,1.273958,1.307639,197521.0,-644.0,-1379.0,-2197.0,-3133.0,-3849.0,-3949.0,-3505.0
30670,2019-12-31,Provence-Alpes-Côte d'Azur,3.0,1.0,1.0,0.0,7.189583,0.093403,0.152083,0.189583,...,0.251042,0.336111,273760.0,54.0,-68.0,-304.0,-569.0,-584.0,-256.0,144.0


### Emissions weigts per region

In [36]:
df_emissions = pd.read_csv("../Data/emissions_2020_2021_reg.csv", sep=";")
df_emissions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24 entries, 0 to 23
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   years                 24 non-null     int64 
 1   Région                24 non-null     object
 2   CO2e (kg) par hab     24 non-null     int64 
 3   Nb hab (en milliers)  24 non-null     object
 4   CO2e (kg) total       24 non-null     object
 5   Poids                 24 non-null     object
dtypes: int64(2), object(4)
memory usage: 1.3+ KB


In [37]:
# Convert the variables to numeric
vars = ["Nb hab (en milliers)", "CO2e (kg) total", "Poids"]

for var in vars:
    df_emissions[var] = df_emissions[var].str.replace(
        r"[,]", ".", regex=True
    )  # convert , into .
    df_emissions[var] = pd.to_numeric(df_emissions[var])


# Obtain the weigts for each region
carbon_weights = df_emissions.groupby(["Région"]).mean()["Poids"]
carbon_weights

Région
Auvergne-Rhône-Alpes          0.120715
Bourgogne-Franche-Comté       0.041468
Bretagne                      0.055237
Centre-Val de Loire           0.037048
Grand Est                     0.080020
Hauts-de-France               0.099611
Normandie                     0.050932
Nouvelle-Aquitaine            0.093203
Occitanie                     0.089981
Pays de la Loire              0.057626
Provence-Alpes-Côte d'Azur    0.084029
Île-de-France                 0.190130
Name: Poids, dtype: float64

### Total emissions per day (MtCO2 / day)

In [38]:
df_carbon_eu = pd.read_csv(
    "../Data/carbonmonitor-eu_datas_2024-02-23.csv", sep=","
)
df_carbon_eu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 345402 entries, 0 to 345401
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   country    345402 non-null  object 
 1   date       345402 non-null  object 
 2   sector     345402 non-null  object 
 3   value      345402 non-null  float64
 4   timestamp  345402 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 13.2+ MB


In [39]:
df_carbon_eu["sector"].unique()

array(['Power', 'Industry', 'Ground Transport', 'Residential',
       'International Aviation', 'Domestic Aviation'], dtype=object)

In [40]:
# Only electricity emissions
df_carbon_fr = df_carbon_eu[df_carbon_eu["sector"] == "Power"]  
# Only France emissions
df_carbon_fr = df_carbon_fr[df_carbon_fr["country"] == "France"]
# Convert to datetime
df_carbon_fr["date"] = pd.to_datetime(df_carbon_fr["date"], format="%d/%m/%Y")
df_carbon_fr = pd.DataFrame(
    {"Date": df_carbon_fr["date"], "Emissions": df_carbon_fr["value"]},
    columns=["Date", "Emissions"],
)
df_carbon_fr = df_carbon_fr.reset_index(drop=True)
df_carbon_fr

,Date,Emissions
0,2019-01-01,0.052318
1,2019-01-02,0.080056
2,2019-01-03,0.094601
3,2019-01-04,0.106710
4,2019-01-05,0.115383
...,...,...
1852,2024-01-27,0.053079
1853,2024-01-28,0.052237
1854,2024-01-29,0.057638
1855,2024-01-30,0.063976


### Synchronize `Date` of `df_X` and `df_carbon_fr`

In [41]:
df_X = df_X[df_X["Date"] >= "2019-01-01"]
df_X = df_X.reset_index(drop=True)
df_X

,Date,Region,saison,week_day,is_holiday,is_bank_holiday,Temp,Temp1,Temp2,Temp3,...,Temp6,Temp7,ce,ce1,ce2,ce3,ce4,ce5,ce6,ce7
0,2019-01-01,Auvergne-Rhône-Alpes,3.0,1.0,1.0,1.0,3.832292,-0.165278,-0.308333,-0.425000,...,-0.602083,-0.593056,398784.0,-546.0,-846.0,-970.0,-1250.0,-2089.0,-3383.0,-4756.0
1,2019-01-01,Bourgogne-Franche-Comté,3.0,1.0,1.0,1.0,6.077083,-0.054861,-0.112500,-0.169792,...,-0.334375,-0.388542,121856.0,-97.0,-92.0,-11.0,139.0,76.0,-182.0,-443.0
2,2019-01-01,Bretagne,3.0,1.0,1.0,1.0,8.797917,0.004861,0.012500,0.019792,...,0.035417,0.042014,129616.0,-228.0,-268.0,-206.0,-204.0,-451.0,-1010.0,-1694.0
3,2019-01-01,Centre-Val de Loire,3.0,1.0,1.0,1.0,7.047917,-0.002778,0.016667,0.029167,...,-0.003125,0.013194,118571.0,-86.0,-88.0,34.0,55.0,-144.0,-411.0,-740.0
4,2019-01-01,Grand Est,3.0,1.0,1.0,1.0,5.632292,0.021528,0.062500,0.086458,...,0.008333,-0.026042,233054.0,-37.0,73.0,258.0,366.0,147.0,-354.0,-1039.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,2019-12-31,Nouvelle-Aquitaine,3.0,1.0,1.0,0.0,5.902083,0.355556,0.700000,0.989583,...,1.493750,1.627083,306332.0,-922.0,-1842.0,-2740.0,-3589.0,-4140.0,-3853.0,-2931.0
4376,2019-12-31,Occitanie,3.0,1.0,1.0,0.0,6.320833,-0.044792,-0.127083,-0.142708,...,0.289583,0.378125,255352.0,-342.0,-778.0,-1235.0,-1786.0,-2176.0,-2255.0,-2175.0
4377,2019-12-31,Pays de la Loire,3.0,1.0,1.0,0.0,5.019792,0.272569,0.543750,0.787500,...,1.273958,1.307639,197521.0,-644.0,-1379.0,-2197.0,-3133.0,-3849.0,-3949.0,-3505.0
4378,2019-12-31,Provence-Alpes-Côte d'Azur,3.0,1.0,1.0,0.0,7.189583,0.093403,0.152083,0.189583,...,0.251042,0.336111,273760.0,54.0,-68.0,-304.0,-569.0,-584.0,-256.0,144.0


In [42]:
df_carbon_fr = df_carbon_fr[df_carbon_fr["Date"] < "2020"]
df_carbon_fr = df_carbon_fr.reset_index(drop=True)
df_carbon_fr

,Date,Emissions
0,2019-01-01,0.052318
1,2019-01-02,0.080056
2,2019-01-03,0.094601
3,2019-01-04,0.106710
4,2019-01-05,0.115383
...,...,...
360,2019-12-27,0.051813
361,2019-12-28,0.054661
362,2019-12-29,0.055831
363,2019-12-30,0.061169


Now `df_X` and `df_carbon_fr` are the same length, modulo the number of regions :

In [43]:
len(df_X) / len(df_carbon_fr)

12.0

In [44]:
REGIONS = np.unique(df_emissions["Région"])
print(len(REGIONS))
print(REGIONS)

12
['Auvergne-Rhône-Alpes' 'Bourgogne-Franche-Comté' 'Bretagne'
 'Centre-Val de Loire' 'Grand Est' 'Hauts-de-France' 'Normandie'
 'Nouvelle-Aquitaine' 'Occitanie' 'Pays de la Loire'
 "Provence-Alpes-Côte d'Azur" 'Île-de-France']


### Apply the weigts to `df_carbon_fr`

In [45]:
df_y = pd.merge(
    df_carbon_fr,
    pd.DataFrame({"Region": REGIONS, "weights": np.array(carbon_weights)}),
    how="cross",
)
df_y["Emissions"] = df_y["Emissions"] * df_y["weights"]
df_y = df_y.drop(columns="weights")
df_y

,Date,Emissions,Region
0,2019-01-01,0.006316,Auvergne-Rhône-Alpes
1,2019-01-01,0.002170,Bourgogne-Franche-Comté
2,2019-01-01,0.002890,Bretagne
3,2019-01-01,0.001938,Centre-Val de Loire
4,2019-01-01,0.004186,Grand Est
...,...,...,...
4375,2019-12-31,0.006660,Nouvelle-Aquitaine
4376,2019-12-31,0.006430,Occitanie
4377,2019-12-31,0.004118,Pays de la Loire
4378,2019-12-31,0.006005,Provence-Alpes-Côte d'Azur


### Add regular and sensitive consumption

In [46]:
# Import data
df_conso = pd.read_csv("../Data/ctr_regions_2019.csv",sep=",")
df_conso

,Date - Heure,Région,c,t,r
0,2019-01-01 00:00:00+01:00,Auvergne-Rhône-Alpes,6971.019467,1067.076870,5903.942596
1,2019-01-01 01:00:00+01:00,Auvergne-Rhône-Alpes,7025.407246,1067.076870,5958.330376
2,2019-01-01 02:00:00+01:00,Auvergne-Rhône-Alpes,7025.407246,1346.699729,5678.707517
3,2019-01-01 03:00:00+01:00,Auvergne-Rhône-Alpes,7317.993960,1359.663584,5958.330376
4,2019-01-01 04:00:00+01:00,Auvergne-Rhône-Alpes,6824.718079,370.408744,6454.309335
...,...,...,...,...,...
105115,2019-12-31 19:00:00+01:00,Île-de-France,6405.857776,432.933342,5972.924434
105116,2019-12-31 20:00:00+01:00,Île-de-France,6256.231402,372.906624,5883.324778
105117,2019-12-31 21:00:00+01:00,Île-de-France,6225.791909,342.467131,5883.324778
105118,2019-12-31 22:00:00+01:00,Île-de-France,6245.250091,434.768338,5810.481752


In [47]:
# We truncate "Date - Heure" so that we only keep the Date
var = "Date - Heure"
res_map = map(truncate, df_conso[var])
date_truncated = pd.Series(np.array(list(res_map)))
date_truncated = pd.to_datetime(date_truncated, format="%Y-%m-%d")
df_conso[var] = date_truncated
df_conso

,Date - Heure,Région,c,t,r
0,2019-01-01,Auvergne-Rhône-Alpes,6971.019467,1067.076870,5903.942596
1,2019-01-01,Auvergne-Rhône-Alpes,7025.407246,1067.076870,5958.330376
2,2019-01-01,Auvergne-Rhône-Alpes,7025.407246,1346.699729,5678.707517
3,2019-01-01,Auvergne-Rhône-Alpes,7317.993960,1359.663584,5958.330376
4,2019-01-01,Auvergne-Rhône-Alpes,6824.718079,370.408744,6454.309335
...,...,...,...,...,...
105115,2019-12-31,Île-de-France,6405.857776,432.933342,5972.924434
105116,2019-12-31,Île-de-France,6256.231402,372.906624,5883.324778
105117,2019-12-31,Île-de-France,6225.791909,342.467131,5883.324778
105118,2019-12-31,Île-de-France,6245.250091,434.768338,5810.481752


In [48]:
renamed_vars = dict(zip(df_conso.columns, key_vars + ['ce_pred','te_pred','re_pred']))

# On renomme les variables
df_conso = df_conso.rename(columns=renamed_vars)

# Il faut sommer la consommation au lieu de la moyenner
df_conso = df_conso.groupby(key_vars).sum()

In [49]:
# On joint les deux datasets
df_X = df_X.set_index(key_vars).join(df_conso, on=key_vars)
df_X

saison  week_day  is_holiday  \
Date       Region                                                     
2019-01-01 Auvergne-Rhône-Alpes           3.0       1.0         1.0   
           Bourgogne-Franche-Comté        3.0       1.0         1.0   
           Bretagne                       3.0       1.0         1.0   
           Centre-Val de Loire            3.0       1.0         1.0   
           Grand Est                      3.0       1.0         1.0   
...                                       ...       ...         ...   
2019-12-31 Nouvelle-Aquitaine             3.0       1.0         1.0   
           Occitanie                      3.0       1.0         1.0   
           Pays de la Loire               3.0       1.0         1.0   
           Provence-Alpes-Côte d'Azur     3.0       1.0         1.0   
           Île-de-France                  3.0       1.0         1.0   

                                       is_bank_holiday      Temp     Temp1  \
Date       Region                                                            
2019-01-01 Auvergne-Rhône-Alpes                    1.0  3.832292 -0.165278   
           Bourgogne-Franche-Comté                 1.0  6.077083 -0.054861   
           Bretagne                                1.0  8.797917  0.004861   
           Centre-Val de Loire                     1.0  7.047917 -0.002778   
           Grand Est                               1.0  5.632292  0.021528   
...                                                ...       ...       ...   
2019-12-31 Nouvelle-Aquitaine                      0.0  5.902083  0.355556   
           Occitanie                               0.0  6.320833 -0.044792   
           Pays de la Loire                        0.0  5.019792  0.272569   
           Provence-Alpes-Côte d'Azur              0.0  7.189583  0.093403   
           Île-de-France                           0.0  0.692708 -0.071528   

                                          Temp2     Temp3     Temp4     Temp5  \
Date       Region                                                               
2019-01-01 Auvergne-Rhône-Alpes       -0.308333 -0.425000 -0.513889 -0.575000   
           Bourgogne-Franche-Comté    -0.112500 -0.169792 -0.225694 -0.280208   
           Bretagne                    0.012500  0.019792  0.025694  0.030208   
           Centre-Val de Loire         0.016667  0.029167  0.025000  0.004167   
           Grand Est                   0.062500  0.086458  0.081250  0.046875   
...                                         ...       ...       ...       ...   
2019-12-31 Nouvelle-Aquitaine          0.700000  0.989583  1.209722  1.360417   
           Occitanie                  -0.127083 -0.142708 -0.056944  0.130208   
           Pays de la Loire            0.543750  0.787500  0.995139  1.166667   
           Provence-Alpes-Côte d'Azur  0.152083  0.189583  0.210417  0.214583   
           Île-de-France              -0.162500 -0.243750 -0.305556 -0.347917   

                                       ...    ce1     ce2     ce3     ce4  \
Date       Region                      ...                                  
2019-01-01 Auvergne-Rhône-Alpes        ... -546.0  -846.0  -970.0 -1250.0   
           Bourgogne-Franche-Comté     ...  -97.0   -92.0   -11.0   139.0   
           Bretagne                    ... -228.0  -268.0  -206.0  -204.0   
           Centre-Val de Loire         ...  -86.0   -88.0    34.0    55.0   
           Grand Est                   ...  -37.0    73.0   258.0   366.0   
...                                    ...    ...     ...     ...     ...   
2019-12-31 Nouvelle-Aquitaine          ... -922.0 -1842.0 -2740.0 -3589.0   
           Occitanie                   ... -342.0  -778.0 -1235.0 -1786.0   
           Pays de la Loire            ... -644.0 -1379.0 -2197.0 -3133.0   
           Provence-Alpes-Côte d'Azur  ...   54.0   -68.0  -304.0  -569.0   
           Île-de-France               ... -113.0  -423.0  -792.0  -963.0   

                                          ce

`ce_pred` is a prediction, it is not the truth. So we take the percentage among `re_pred` and `te_pred`, and we apply those percentages to `ce`, giving new variables `re` and `te`. We add those variables to the list of predictive variables called `values_var`.

In [50]:
values_var += ['te','re']

df_X['te_pred'] = df_X['te_pred'] / df_X['ce_pred']
df_X['re_pred'] = df_X['re_pred'] / df_X['ce_pred']
df_X['te'] = df_X['te_pred'] * df_X['ce']
df_X['re'] = df_X['re_pred'] * df_X['ce']
df_X = df_X.drop(columns=['ce_pred','te_pred','re_pred'])
df_X

saison  week_day  is_holiday  \
Date       Region                                                     
2019-01-01 Auvergne-Rhône-Alpes           3.0       1.0         1.0   
           Bourgogne-Franche-Comté        3.0       1.0         1.0   
           Bretagne                       3.0       1.0         1.0   
           Centre-Val de Loire            3.0       1.0         1.0   
           Grand Est                      3.0       1.0         1.0   
...                                       ...       ...         ...   
2019-12-31 Nouvelle-Aquitaine             3.0       1.0         1.0   
           Occitanie                      3.0       1.0         1.0   
           Pays de la Loire               3.0       1.0         1.0   
           Provence-Alpes-Côte d'Azur     3.0       1.0         1.0   
           Île-de-France                  3.0       1.0         1.0   

                                       is_bank_holiday      Temp     Temp1  \
Date       Region                                                            
2019-01-01 Auvergne-Rhône-Alpes                    1.0  3.832292 -0.165278   
           Bourgogne-Franche-Comté                 1.0  6.077083 -0.054861   
           Bretagne                                1.0  8.797917  0.004861   
           Centre-Val de Loire                     1.0  7.047917 -0.002778   
           Grand Est                               1.0  5.632292  0.021528   
...                                                ...       ...       ...   
2019-12-31 Nouvelle-Aquitaine                      0.0  5.902083  0.355556   
           Occitanie                               0.0  6.320833 -0.044792   
           Pays de la Loire                        0.0  5.019792  0.272569   
           Provence-Alpes-Côte d'Azur              0.0  7.189583  0.093403   
           Île-de-France                           0.0  0.692708 -0.071528   

                                          Temp2     Temp3     Temp4     Temp5  \
Date       Region                                                               
2019-01-01 Auvergne-Rhône-Alpes       -0.308333 -0.425000 -0.513889 -0.575000   
           Bourgogne-Franche-Comté    -0.112500 -0.169792 -0.225694 -0.280208   
           Bretagne                    0.012500  0.019792  0.025694  0.030208   
           Centre-Val de Loire         0.016667  0.029167  0.025000  0.004167   
           Grand Est                   0.062500  0.086458  0.081250  0.046875   
...                                         ...       ...       ...       ...   
2019-12-31 Nouvelle-Aquitaine          0.700000  0.989583  1.209722  1.360417   
           Occitanie                  -0.127083 -0.142708 -0.056944  0.130208   
           Pays de la Loire            0.543750  0.787500  0.995139  1.166667   
           Provence-Alpes-Côte d'Azur  0.152083  0.189583  0.210417  0.214583   
           Île-de-France              -0.162500 -0.243750 -0.305556 -0.347917   

                                       ...        ce    ce1     ce2     ce3  \
Date       Region                      ...                                    
2019-01-01 Auvergne-Rhône-Alpes        ...  398784.0 -546.0  -846.0  -970.0   
           Bourgogne-Franche-Comté     ...  121856.0  -97.0   -92.0   -11.0   
           Bretagne                    ...  129616.0 -228.0  -268.0  -206.0   
           Centre-Val de Loire         ...  118571.0  -86.0   -88.0    34.0   
           Grand Est                   ...  233054.0  -37.0    73.0   258.0   
...                                    ...       ...    ...     ...     ...   
2019-12-31 Nouvelle-Aquitaine          ...  306332.0 -922.0 -1842.0 -2740.0   
           Occitanie                   ...  255352.0 -342.0  -778.0 -1235.0   
           Pays de la Loire            ...  197521.0 -644.0 -1379.0 -2197.0   
           Provence-Alpes-Côte d'Azur  ...  273760.0   54.0   -68.0  -304.0   
           Île-de-France               ...  504624.0 -113.0  -423.0  -792.0   

                  

### Final merge

In [51]:
df_processed = df_X.join(df_y.set_index(key_vars), on=key_vars)
df_processed = df_processed.reset_index()
df_processed

,Date,Region,saison,week_day,is_holiday,is_bank_holiday,Temp,Temp1,Temp2,Temp3,...,ce1,ce2,ce3,ce4,ce5,ce6,ce7,te,re,Emissions
0,2019-01-01,Auvergne-Rhône-Alpes,3.0,1.0,1.0,1.0,3.832292,-0.165278,-0.308333,-0.425000,...,-546.0,-846.0,-970.0,-1250.0,-2089.0,-3383.0,-4756.0,59234.225607,339549.774393,0.006316
1,2019-01-01,Bourgogne-Franche-Comté,3.0,1.0,1.0,1.0,6.077083,-0.054861,-0.112500,-0.169792,...,-97.0,-92.0,-11.0,139.0,76.0,-182.0,-443.0,15881.572707,105974.427293,0.002170
2,2019-01-01,Bretagne,3.0,1.0,1.0,1.0,8.797917,0.004861,0.012500,0.019792,...,-228.0,-268.0,-206.0,-204.0,-451.0,-1010.0,-1694.0,5400.430173,124215.569827,0.002890
3,2019-01-01,Centre-Val de Loire,3.0,1.0,1.0,1.0,7.047917,-0.002778,0.016667,0.029167,...,-86.0,-88.0,34.0,55.0,-144.0,-411.0,-740.0,11273.021319,107297.978681,0.001938
4,2019-01-01,Grand Est,3.0,1.0,1.0,1.0,5.632292,0.021528,0.062500,0.086458,...,-37.0,73.0,258.0,366.0,147.0,-354.0,-1039.0,33262.258946,199791.741054,0.004186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4375,2019-12-31,Nouvelle-Aquitaine,3.0,1.0,1.0,0.0,5.902083,0.355556,0.700000,0.989583,...,-922.0,-1842.0,-2740.0,-3589.0,-4140.0,-3853.0,-2931.0,25508.889143,280823.110857,0.006660
4376,2019-12-31,Occitanie,3.0,1.0,1.0,0.0,6.320833,-0.044792,-0.127083,-0.142708,...,-342.0,-778.0,-1235.0,-1786.0,-2176.0,-2255.0,-2175.0,30498.494232,224853.505768,0.006430
4377,2019-12-31,Pays de la Loire,3.0,1.0,1.0,0.0,5.019792,0.272569,0.543750,0.787500,...,-644.0,-1379.0,-2197.0,-3133.0,-3849.0,-3949.0,-3505.0,26746.540296,170774.459704,0.004118
4378,2019-12-31,Provence-Alpes-Côte d'Azur,3.0,1.0,1.0,0.0,7.189583,0.093403,0.152083,0.189583,...,54.0,-68.0,-304.0,-569.0,-584.0,-256.0,144.0,19794.159517,253965.840483,0.006005


## Save dataset

In [52]:
integer_vars = ['saison', 'week_day', 'is_holiday', 'is_bank_holiday']
df_processed[integer_vars] = df_processed[integer_vars].astype(int)

In [53]:
df_processed.dtypes

Date               datetime64[ns]
Region                     object
saison                      int32
week_day                    int32
is_holiday                  int32
is_bank_holiday             int32
Temp                      float64
Temp1                     float64
Temp2                     float64
Temp3                     float64
Temp4                     float64
Temp5                     float64
Temp6                     float64
Temp7                     float64
ce                        float64
ce1                       float64
ce2                       float64
ce3                       float64
ce4                       float64
ce5                       float64
ce6                       float64
ce7                       float64
te                        float64
re                        float64
Emissions                 float64
dtype: object

In [54]:
df_processed.to_csv('../Data/carbon_data.csv', index=False)